In [11]:
import requests, time
from bs4 import BeautifulSoup

In [14]:
ptt_url = 'https://www.ptt.cc'
# setting url and headers
url = "https://www.ptt.cc/bbs/NBA/index.html"
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}


In [35]:
def get_url(url):
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    return soup


def last_url(soup):
    if 'disabled' in soup.select('div.btn-group a.btn')[3]['class']:
        print('All pages have been catched !!')
    else:
        last_page = ptt_url + soup.select('div.btn-group a.btn')[3]['href']
    

def list_url(soup):
    try:
        for url in soup.select('div.btn-group btn-group-paging a.btn'):
            ptt_urls = soup.select('div.title a')
            print(ptt_url + url['href'])
    except Exception as e:
        print(e)
        
        
def search_bar(input):
    pass
    

In [36]:
soup = get_url('https://www.ptt.cc/bbs/NBA/search?q=wade')
list_url(soup)

In [37]:
soup

<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>wade - NBA - 批踢踢實業坊</title>
<link href="//images.ptt.cc/bbs/v2.27/bbs-common.css" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.27/bbs-base.css" media="screen" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.27/bbs-custom.css" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.27/pushstream.css" media="screen" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.27/bbs-print.css" media="print" rel="stylesheet" type="text/css"/>
</head>
<body>
<div id="topbar-container">
<div class="bbs-content" id="topbar">
<a href="/bbs/" id="logo">批踢踢實業坊</a>
<span>›</span>
<a class="board" href="/bbs/NBA/index.html"><span class="board-label">看板 </span>NBA</a>
<a class="right small" href="/about.html">關於我們</a>
<a class="right small" href="/contact.html">聯絡資訊</a>
</div>
</div>
<div id="main-c